In [1]:
!pip install torch

In [2]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

* We support Llama, Mistral, Phi-3, Gemma, Yi, DeepSeek, Qwen, TinyLlama, Vicuna, Open Hermes etc
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
* [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)
* [**NEW**] We make Mistral NeMo 12B 2x faster and fit in under 12GB of VRAM! [Mistral NeMo notebook](https://colab.research.google.com/drive/17d3U-CAIwzmbDRqbZ9NnpHxCkmXB6LZ0?usp=sharing)

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.7: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.10.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
!pip install datasets

In [6]:
from datasets import load_dataset
dataset = load_dataset('yuan-yang/MALLS-v0')
print(dataset.column_names)

README.md: 0.00B [00:00, ?B/s]

MALLS-v0.1-train.json: 0.00B [00:00, ?B/s]

MALLS-v0.1-test.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/27284 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'train': ['FOL', 'NL'], 'test': ['FOL', 'NL']}


In [7]:
malls_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    # Create a static instruction for all examples.
    instruction_text = "Translate the following natural language statements into their corresponding first-order logic representations."
    nls = examples["NL"]
    fols  = examples["FOL"]
    texts = []

    for fol, nl in zip(fols, nls):
        text = malls_prompt.format(instruction_text, nl, fol) + EOS_TOKEN
        texts.append(text)

    # Tokenize the formatted text
    tokenized_output = tokenizer(texts, truncation=True, padding="max_length", max_length=512)  # Adjust max_length as needed
    return tokenized_output

# Apply formatting and tokenization
dataset = dataset.map(formatting_prompts_func, batched=True)

# Check dataset structure
print(dataset["train"].column_names)
# Check dataset structure
print(dataset["test"].column_names)

Map:   0%|          | 0/27284 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

['FOL', 'NL', 'input_ids', 'attention_mask']
['FOL', 'NL', 'input_ids', 'attention_mask']


<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset["train"],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 45,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [9]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
6.971 GB of memory reserved.


In [10]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 27,284 | Num Epochs = 1 | Total steps = 45
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,9.819600
2,9.892300
3,9.866100
4,9.704100
5,9.801800
6,9.346700
7,9.279100
8,8.100000
9,7.655800
10,6.975700


In [11]:
dataset['test'].column_names

['FOL', 'NL', 'input_ids', 'attention_mask']

In [12]:
import pandas as pd
import plotly.express as px

# Access the log history (a list of dictionaries) from the Trainer's state
log_history = trainer.state.log_history
print(log_history)


[{'loss': 9.8196, 'grad_norm': nan, 'learning_rate': 0.0, 'epoch': 0.00029321213898255387, 'step': 1}, {'loss': 9.8923, 'grad_norm': nan, 'learning_rate': 0.0, 'epoch': 0.0005864242779651077, 'step': 2}, {'loss': 9.8661, 'grad_norm': nan, 'learning_rate': 0.0, 'epoch': 0.0008796364169476617, 'step': 3}, {'loss': 9.7041, 'grad_norm': 27.694047927856445, 'learning_rate': 0.0, 'epoch': 0.0011728485559302155, 'step': 4}, {'loss': 9.8018, 'grad_norm': 29.6751766204834, 'learning_rate': 4e-05, 'epoch': 0.0014660606949127694, 'step': 5}, {'loss': 9.3467, 'grad_norm': nan, 'learning_rate': 8e-05, 'epoch': 0.0017592728338953233, 'step': 6}, {'loss': 9.2791, 'grad_norm': 24.587417602539062, 'learning_rate': 8e-05, 'epoch': 0.002052484972877877, 'step': 7}, {'loss': 8.1, 'grad_norm': 6.153500556945801, 'learning_rate': 0.00012, 'epoch': 0.002345697111860431, 'step': 8}, {'loss': 7.6558, 'grad_norm': 5.480885982513428, 'learning_rate': 0.00016, 'epoch': 0.002638909250842985, 'step': 9}, {'loss': 6

In [13]:

# Convert log history to a DataFrame
df = pd.DataFrame(log_history)

# Check if the necessary columns exist
if 'loss' in df.columns and 'step' in df.columns:
    # Create a line plot of loss over training steps
    fig = px.line(df, x='step', y='loss', title='Training Loss Over Time')
    fig.show()
else:
    print("The log history does not contain 'loss' or 'step' data.")



In [14]:
import pandas as pd

# Pre-enable faster inference if it doesn't need to be done every iteration.
FastLanguageModel.for_inference(model)

results = []
TOTAL = len(dataset['test'])

for i, example in enumerate(dataset['test'], start=1):
    nl = example['NL']
    fol = example['FOL']

    # Prepare prompt text using the formatted prompt.
    prompt_text = malls_prompt.format(
        "Translate the following natural language statements into their corresponding first-order logic representations.",
        nl,
        ""
    )

    inputs = tokenizer([prompt_text], return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
    decoded_text = tokenizer.batch_decode(outputs)[0]

    # Find the start of the response section
    start_marker = "### Response:"
    start_idx = decoded_text.find(start_marker)
    if start_idx != -1:
        response_text = decoded_text[start_idx + len(start_marker):].strip()
        # Optionally remove the end-of-text token
        end_marker = "<|end_of_text|>"
        end_idx = response_text.find(end_marker)
        if end_idx != -1:
            response_text = response_text[:end_idx].strip()
    else:
        response_text = '<empty>'

    results.append({'NL': nl, 'FOL': fol, 'Response': response_text})
    print(f"{i}/{TOTAL} examples processed")

# Create DataFrame once from the accumulated results
prediction_result_test = pd.DataFrame(results)
prediction_result_test.head()


1/1000 examples processed
2/1000 examples processed
3/1000 examples processed
4/1000 examples processed
5/1000 examples processed
6/1000 examples processed
7/1000 examples processed
8/1000 examples processed
9/1000 examples processed
10/1000 examples processed
11/1000 examples processed
12/1000 examples processed
13/1000 examples processed
14/1000 examples processed
15/1000 examples processed
16/1000 examples processed
17/1000 examples processed
18/1000 examples processed
19/1000 examples processed
20/1000 examples processed
21/1000 examples processed
22/1000 examples processed
23/1000 examples processed
24/1000 examples processed
25/1000 examples processed
26/1000 examples processed
27/1000 examples processed
28/1000 examples processed
29/1000 examples processed
30/1000 examples processed
31/1000 examples processed
32/1000 examples processed
33/1000 examples processed
34/1000 examples processed
35/1000 examples processed
36/1000 examples processed
37/1000 examples processed
38/1000 ex

,NL,FOL,Response
0,A vacation is relaxing if it includes beautifu...,∀x (Vacation(x) ∧ Relaxing(x) → (BeautifulScen...,∀x (Vacation(x) ∧ IncludesBeautifulScenery(x) ...
1,"A gemstone can be a diamond, a ruby, or an eme...",∀x (Gemstone(x) → ((Diamond(x) ∧ ¬(Ruby(x) ∨ E...,∀x (Gemstone(x) → (Diamond(x) ∨ Ruby(x) ∨ Emer...
2,People who speak more than one language are bi...,∀x (SpeakMultipleLanguages(x) → (Bilingual(x) ...,∀x (Person(x) ∧ SpeaksMultipleLanguages(x) → (...
3,An organization is non-profit if it has a char...,∀x (Organization(x) ∧ CharitableMission(x) ∧ ¬...,∀x (Organization(x) ∧ CharitableMission(x) ∧ N...
4,A flower is characterized as a monocot if it h...,∀x (Flower(x) ∧ SingleCotyledon(x) ∧ ParallelV...,∀x (Flower(x) ∧ Monocot(x) ↔ (SingleCotyledon(...


In [ ]:
# prediction_result_test.to_csv('prediction_result_test.csv', index=False)

In [15]:
prediction_result_test.head()

,NL,FOL,Response
0,A vacation is relaxing if it includes beautifu...,∀x (Vacation(x) ∧ Relaxing(x) → (BeautifulScen...,∀x (Vacation(x) ∧ IncludesBeautifulScenery(x) ...
1,"A gemstone can be a diamond, a ruby, or an eme...",∀x (Gemstone(x) → ((Diamond(x) ∧ ¬(Ruby(x) ∨ E...,∀x (Gemstone(x) → (Diamond(x) ∨ Ruby(x) ∨ Emer...
2,People who speak more than one language are bi...,∀x (SpeakMultipleLanguages(x) → (Bilingual(x) ...,∀x (Person(x) ∧ SpeaksMultipleLanguages(x) → (...
3,An organization is non-profit if it has a char...,∀x (Organization(x) ∧ CharitableMission(x) ∧ ¬...,∀x (Organization(x) ∧ CharitableMission(x) ∧ N...
4,A flower is characterized as a monocot if it h...,∀x (Flower(x) ∧ SingleCotyledon(x) ∧ ParallelV...,∀x (Flower(x) ∧ Monocot(x) ↔ (SingleCotyledon(...


In [16]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

492.2389 seconds used for training.
8.2 minutes used for training.
Peak reserved memory = 7.281 GB.
Peak reserved memory for training = 0.31 GB.
Peak reserved memory % of max memory = 49.393 %.
Peak reserved memory for training % of max memory = 2.103 %.


# SAVE THE MODEL

In [17]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

# DEMO

In [18]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    malls_prompt.format(
        "Translate the following natural language statements into their corresponding first-order logic representations", # instruction
        "Some dog is a pet", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Translate the following natural language statements into their corresponding first-order logic representations

### Input:
Some dog is a pet

### Response:
∃x (Dog(x) ∧ Pet(x))<|end_of_text|>
